In [11]:
from os import path, makedirs
from itertools import product
import numpy as np

from brainpipe.system import study

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

In [14]:
"""
Compute classification of High and Low condition
using diagonal RSA matrices (2-120Hz)
"""

st = study('Olfacto')
rsa_path = path.join(st.path,'feature/1_RSA_diag_{}_by_cond/')
rsa_name = path.join(rsa_path, 'rsa_diag_{}_cond_{}.npz')
path_save = path.join(st.path, 'classified/RSA_clf_diag_time_{}_by_cond/')
clf_name = path.join(path_save, '{}_LDA_clf_rsa_diag_low_high.npz')

phases, conds = ['E','R','E_R'], ['low','high']
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
nperm = 1000

for su, ph in product(subjects,phases):
    mat0 = np.load(rsa_name.format(ph,su,conds[0]), allow_pickle=True)
    mat1 = np.load(rsa_name.format(ph,su,conds[1]))['rsa']
    
    rsa_list = [mat0['rsa'], mat1]
    nelecs, _, nsamples = mat0['rsa'].shape
    print (su, mat0.files, 'rsa shape: ', [rsa.shape for rsa in rsa_list])

    name_classif = clf_name.format(ph,su)
    if not path.exists(name_classif):
        kwargs = {}
        kwargs['labels'], kwargs['channels'] = mat0['labels'], mat0['channels']
        kwargs['xyz'], kwargs['time'] = mat0['xyz'], mat0['time']

        permut,auc = np.zeros((nelecs,nsamples,nperm)),np.zeros((nelecs,nsamples))
        for elec_num in range(nelecs):
            print('--» processing',su, 'elec', elec_num,'/',nelecs)
            data_elec = [x[elec_num] for x in rsa_list]

            # create a data matrix, concatenate along the trial dimension
            x = np.concatenate(data_elec, axis=0)
            print ('Size of the concatenated data: ', x.shape)
            y = np.hstack([np.array([i]*len(x)) for i, x in enumerate(data_elec)])
            print ('Size of label for classif: ', len(y))

            for t in range(nsamples):
                X_t = x[:,t][:,np.newaxis]
                clf = LDA()
                cv = SKFold(10)
                score, permutation_scores, pvalue = permutation_test_score(
                clf, X_t, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                permut[elec_num,t] += permutation_scores
                auc[elec_num,t] += score
#                     print("clf score for %t %s (pvalue : %s)" % (t, score, pvalue))
        kwargs['perm'], kwargs['auc'] = permut, auc
    
        #Save plots
        if not path.exists(path_save.format(ph)):
            makedirs(path_save.format(ph))
        np.savez(name_classif, **kwargs)
        
        del x, y, data_elec, permutation_scores, score, rsa_list
    else:
        print(name_classif, 'already computed')

-> Olfacto loaded
LEFC ['channels', 'xyz', 'time', 'labels', 'rsa'] rsa shape:  [(36, 561, 61), (36, 210, 61)]
--» processing LEFC elec 0 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 1 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 2 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 3 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 4 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 5 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 6 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 7 / 36
Size of the concatenated data:  (771, 61)
Size of label for classif:  771
--» processing LEFC elec 8 / 36
Size of t

--» processing LEFC elec 3 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 4 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 5 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 6 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 7 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 8 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 9 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 10 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 11 / 36
Size of the concatenated data:  (660, 61)
Size of label for classif:  660
--» processing LEFC elec 12 / 36
Size of the

--» processing CHAF elec 43 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 44 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 45 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 46 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 47 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 48 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 49 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 50 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 51 / 57
Size of the concatenated data:  (120, 61)
Size of label for classif:  120
--» processing CHAF elec 52 / 57
Size

--» processing CHAF elec 4 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 5 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 6 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 7 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 8 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 9 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 10 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 11 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 12 / 57
Size of the concatenated data:  (252, 61)
Size of label for classif:  252
--» processing CHAF elec 13 / 57
Size of th

--» processing VACJ elec 23 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 24 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 25 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 26 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 27 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 28 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 29 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 30 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 31 / 48
Size of the concatenated data:  (110, 61)
Size of label for classif:  110
--» processing VACJ elec 32 / 48
Size

...

--» processing SEMC elec 44 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 45 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 46 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 47 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 48 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 49 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 50 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 51 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 52 / 62
Size of the concatenated data:  (141, 61)
Size of label for classif:  141
--» processing SEMC elec 53 / 62
Size

--» processing SEMC elec 58 / 62
Size of the concatenated data:  (546, 61)
Size of label for classif:  546
--» processing SEMC elec 59 / 62
Size of the concatenated data:  (546, 61)
Size of label for classif:  546
--» processing SEMC elec 60 / 62
Size of the concatenated data:  (546, 61)
Size of label for classif:  546
--» processing SEMC elec 61 / 62
Size of the concatenated data:  (546, 61)
Size of label for classif:  546
FERJ ['channels', 'xyz', 'time', 'labels', 'rsa'] rsa shape:  [(45, 136, 61), (45, 91, 61)]
--» processing FERJ elec 0 / 45
Size of the concatenated data:  (227, 61)
Size of label for classif:  227
--» processing FERJ elec 1 / 45
Size of the concatenated data:  (227, 61)
Size of label for classif:  227
--» processing FERJ elec 2 / 45
Size of the concatenated data:  (227, 61)
Size of label for classif:  227
--» processing FERJ elec 3 / 45
Size of the concatenated data:  (227, 61)
Size of label for classif:  227
--» processing FERJ elec 4 / 45
Size of the concatenated

--» processing FERJ elec 27 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 28 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 29 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 30 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 31 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 32 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 33 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 34 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 35 / 45
Size of the concatenated data:  (132, 61)
Size of label for classif:  132
--» processing FERJ elec 36 / 45
Size

--» processing PIRJ elec 13 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 14 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 15 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 16 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 17 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 18 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 19 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 20 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 21 / 30
Size of the concatenated data:  (146, 61)
Size of label for classif:  146
--» processing PIRJ elec 22 / 30
Size

--» processing PIRJ elec 29 / 30
Size of the concatenated data:  (400, 61)
Size of label for classif:  400
